# Entrono #

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.ensemble import GradientBoostingRegressor
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import PyPDFLoader


In [28]:
# ----------------------------
# Pub/Sub Broker
# ----------------------------
class PubSubBroker:
    def __init__(self):
        self.subscribers = []

    def subscribe(self, callback):
        self.subscribers.append(callback)

    def publish(self, data):
        for callback in self.subscribers:
            callback(data)

# ----------------------------
# Whiteboard (memoria compartida)
# ----------------------------
class Whiteboard:
    def __init__(self):
        self.data = {}

    def write(self, key, value):
        self.data[key] = value

    def read(self):
        while len(self.data) < 2:
            print("Esperando datos en la pizarra... ", self.data)
            time.sleep(5)
        return dict(self.data)

    def clear(self):
        self.data = {}

# ----------------------------
# DataFetcher (Insertar agente de recolección de datos)
# ----------------------------
class DataFetcher:
    def __init__(self, dataframe):
        self.df = dataframe
        self.index = 0

    def has_next(self):
        return self.index < len(self.df)
    
    def get_next_ticker(self):
        if self.index < len(self.df):
            return self.df.iloc[self.index]['Ticker']
        return None

    def fetch_data(self):
        current = self.df.loc[self.index]
        self.index += 1
        return current
# ----------------------------

In [29]:
class traditionalAgent:
    def __init__(self, broker, whiteboard, data_fetcher):
        self.broker = broker
        self.whiteboard = whiteboard
        self.data_fetcher = data_fetcher
        self.df = None
        self.broker.subscribe(self.group_traditional)

    def set_dataframe(self, df):
        self.df = df

    def group_traditional(self, data):
        idx = self.data_fetcher.index
        if idx < 2 or self.df is None:
            self.whiteboard.write('trad_pred', data['Close'])
            return

        # Filtrar histórico por ticker actual
        df_hist = self.df.iloc[:idx].copy()
        df_hist = df_hist[df_hist['Ticker'] == data['Ticker']].reset_index(drop=True)

        if len(df_hist) < 10:
            self.whiteboard.write('trad_pred', data['Close'])
            return

        # Columnas predictoras
        features = ['Open', 'High', 'Low', 'Volume', 'RSI', 'SMA_20', 'MACD']

        # Objetivo: Close futuro (t+1)
        df_hist['Close_Future'] = df_hist['Close'].shift(-1)
        df_hist = df_hist.fillna(method='ffill')

        try:
            X = df_hist[features]
            y = df_hist['Close_Future']
            current_features = np.array([data[f] for f in features]).reshape(1, -1)

            # pred1: Regresión lineal multivariable
            model1 = LinearRegression()
            model1.fit(X, y)
            pred1 = model1.predict(current_features)[0]

            # pred2: Gradient Boosting multivariable
            model2 = GradientBoostingRegressor()
            model2.fit(X, y)
            pred2 = model2.predict(current_features)[0]

        except Exception:
            pred1 = pred2 = data['Close']

        avg = (pred1 + pred2) / 2
        self.whiteboard.write('trad_pred', avg)

In [30]:
class AgenteIA:
    def __init__(self, broker, whiteboard, data_fetcher):
        self.broker = broker
        self.whiteboard = whiteboard
        #self.broker.subscribe(self.group_ai)
        self.data_fetcher = data_fetcher

        # Inicializa embeddings y base de datos vectorial
        self.embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
        self.db = None  # Se cargará después con documentos reales

    def cargar_documentos(self, ruta):
        documentos = []
        for archivo in os.listdir(ruta):
            if archivo.endswith(".txt"):
                documentos.extend(TextLoader(os.path.join(ruta, archivo)).load())
            elif archivo.endswith(".pdf"):
                documentos.extend(PyPDFLoader(os.path.join(ruta, archivo)).load())
        return documentos

    def dividir_chunks(self, documentos):
        splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        return splitter.split_documents(documentos)

    def crear_o_cargar_chromadb(self, chunks, persist_directory="db_finanzas"):
        if not os.path.exists(persist_directory):
            self.db = Chroma.from_documents(chunks, self.embedding_function, persist_directory=persist_directory)
            self.db.persist()
        else:
            self.db = Chroma(persist_directory=persist_directory, embedding_function=self.embedding_function)

    def consulta_lmstudio(self, prompt):
        url = "http://localhost:1234/v1/chat/completions"
        headers = {"Content-Type": "application/json"}

        data = {
            "messages": [{
                "role": "system",
                "content": "Eres un asistente que responde SOLO con JSON, nunca expliques, nunca incluyas texto adicional.\n" + prompt
            }],
            "temperature": 0.4,
            "max_tokens": 1048
        }

        response = requests.post(url, headers=headers, json=data)
        return response.json()["choices"][0]["message"]["content"]

    def consulta_rag(self, pregunta):
        if self.db is None:
            raise Exception("La base de datos vectorial no ha sido cargada.")
        similares = self.db.similarity_search(pregunta, k=3)
        contexto = "\n\n".join([doc.page_content for doc in similares])

        prompt = f"""Con base en el siguiente contexto, responde la pregunta solo en formato JSON válido con claves 'fecha' y 'valor':

        Contexto:
        {contexto}

        Pregunta: {pregunta}
        """
        res = self.consulta_lmstudio(prompt)
        return res

    def group_ai(self, data):
        stock_symbol = data.get('Ticker')
        current_date = pd.to_datetime(data['Datetime'])
        df_stock = self.data_fetcher.df
        df_filtrado = df_stock[(df_stock['Ticker'] == stock_symbol) & (pd.to_datetime(df_stock['Datetime']) < current_date)]
        if df_filtrado.empty or current_date < pd.to_datetime("2025-07-13"):
            self.whiteboard.write('ai_pred', data['Close'])  # fallback
            return
        df_historico = df_filtrado.sort_values(by='Datetime')
        texto_contexto_historico = df_historico.to_string(index=False)
        if self.data_fetcher.index > 1:
            last_close = self.data_fetcher.df.loc[self.data_fetcher.index - 2]['Close']
        else:
            last_close = data['Close']
            
        pregunta = f"""Dado el siguiente historial de precios del activo {stock_symbol}, ¿cuál sería una predicción razonable del valor de cierre para el día y la hora {data['Datetime']}?
        
        Historial:
        {texto_contexto_historico}
        
        Responde en formato JSON, un solo elemento con la siguiente estructura: {{ "fecha": "AAAA-MM-DD hh:mm ", "valor": <número> }}"""
        try:
            respuesta = self.consulta_rag(pregunta)
            resultado = eval(respuesta)
            print(resultado)
            print("----------------------------")
            avg = resultado['valor']
        except Exception as e:
            print("Error en predicción:", e)
            avg = last_close  # fallback
        self.whiteboard.read
        self.whiteboard.write('ai_pred', avg)

In [35]:
# ----------------------------
# MarketEnvironment
# ----------------------------
class MarketEnvironment:
    def __init__(self, df):
        self.df = df
        self.data_fetcher = DataFetcher(df)
        self.broker = PubSubBroker()
        self.whiteboard = Whiteboard()
        self.histories = {}  # Diccionario para almacenar cada empresa

        # Agentes de predicción
        self.agentsTraditional = traditionalAgent(self.broker, self.whiteboard, self.data_fetcher)
        self.agentsIA = AgenteIA(self.broker, self.whiteboard, self.data_fetcher)
        self.agentsTraditional.set_dataframe(df)

        documentos = self.agentsIA.cargar_documentos("documentos")
        chunks = self.agentsIA.dividir_chunks(documentos)
        self.agentsIA.crear_o_cargar_chromadb(chunks)
        
        # Suscribir los grupos de agentes al broker
        self.broker.subscribe(self.agentsTraditional.group_traditional)
        self.broker.subscribe(self.agentsIA.group_ai)

    def run(self):
        tickers = self.df['Ticker'].unique().tolist()

        for ticker in tickers:
            print(f"Iniciando simulación para {ticker}...")

            df_ticker = self.df[self.df['Ticker'] == ticker].reset_index(drop=True)
            self.data_fetcher = DataFetcher(df_ticker)
            self.agentsTraditional.set_dataframe(df_ticker)

            self.histories[ticker] = pd.DataFrame(columns=['Datetime', 'Real_Close', 'Trad_Prediction', 'AI_Prediction'])

            # Primer paso: hacer la primera predicción (usando t)
            if not self.data_fetcher.has_next():
                continue
            current_data = self.data_fetcher.fetch_data()
            self.whiteboard.clear()
            self.broker.publish(current_data)
            last_prediction = self.whiteboard.read()  # ← esto reemplaza a last_data

            while self.data_fetcher.has_next():
                current_data = self.data_fetcher.fetch_data()

                # Guardar la predicción previa (hecha en t) contra el valor real de t+1
                self.histories[ticker].loc[len(self.histories[ticker])] = [
                    current_data['Datetime'],
                    current_data['Close'],
                    last_prediction.get('trad_pred', current_data['Close']),
                    last_prediction.get('ai_pred', current_data['Close'])
                ]

                #print(
                #    f"[{current_data['Datetime']}] {ticker} | "
                #    f"Real: {current_data['Close']:.2f} | "
                #    f"Tradicional: {last_prediction.get('trad_pred', current_data['Close']):.2f} | "
                #    f"IA: {last_prediction.get('ai_pred', current_data['Close']):.2f}"
                #)

                # Hacer predicción para t+2 usando datos de t+1
                self.whiteboard.clear()
                self.broker.publish(current_data)
                last_prediction = self.whiteboard.read()

            self.plot_graph(ticker)
            #self.histories[ticker].to_excel(f"simulacion_{ticker}.xlsx", index=False)
            print(f"✔ Terminó {ticker}")


    def plot_graph(self, ticker):
        history = self.histories[ticker]

        # Ignorar el primer punto
        history = history.iloc[1:]

        plt.figure(figsize=(12,5))
        plt.plot(history['Datetime'], history['Real_Close'], label='Precio Real', marker='o')
        plt.plot(history['Datetime'], history['Trad_Prediction'], label='Predicción Tradicional', linestyle='--', marker='x')
        plt.plot(history['Datetime'], history['AI_Prediction'], label='Predicción IA', linestyle='-.', marker='s')
        plt.legend()
        plt.title(f'Simulación Multiagente - {ticker}')
        plt.xlabel('Tiempo')
        plt.ylabel('Precio de Cierre')
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.show()


In [37]:
# ----------------------------
# Main
# ----------------------------
if __name__ == "__main__":
    # Cargar datos
    df = pd.read_excel("historico_top10_indicadores_completos.xlsx")
    env = MarketEnvironment(df)
    env.run()

NameError: name 'PyPDFLoader' is not defined